# CSE 5243 - Introduction to Data Mining
## Homework 5: Association Analysis
- Semester: Spring 2023
- Instructor:  Tom Bihari
- Section: Wednesday/Friday 9:35AM 
- Student Name: Jiyong Kwag
- Student Email: kwag.3@buckeyemail.osu.edu
- Student ID: 500165290

Template Version V1.
***

# Introduction

### Objectives

In this lab, you will use the "AssociationsGroceriesDataV1.xlsx" dataset provided on Carmen to find potential association rules.

The objectives of this assignment are:
- Practice the Association Analysis content we covered this semester.
- Understand “why” the particular topics, techniques, etc., are important from a practical perspective.
- Understand how to choose and use appropriate tools to solve the provided problems.

### The Dataset
- This workbook contains is a market basket dataset containing transactions.
- The data file captures the data in "long format". Specifically, every row corresponds to the transaction id and the item. If the specific transaction id has multiple items, there will be multiple rows in the data.
- You can process the data however you like, but it is recommended you convert into a one-hot-encoded data structure. This will allow you to easily use the mlxtend package.

## The Business Problem
- Assume this dataset contains all of the transactions for one month for our store.  We wish to find association rules that would improve our revenue as follows:
  - We would discount one of our products by 10% each month, with the hope that this would encourage customers to visit our store to purchse that product 5% more frequently, and also purchase  other products (that are not discounted) more frequently.
- Practically speaking, we would like to come up with **two-item** rules (one antecedent and one consequent: (A -> B)) and choose the one that best adds to our revenues  (based on the rule support, confidence, etc.).

### Proper Answers
- To make everyone's lives a little easier, when writing Itemsets and Rules, please list the items in **lexagraphical order**: {A}, {B}, {A,B}, {A,C}, {A,B,C},…, {A,B,C}->{D}, {A,B}->{C,D}
    - If you have trouble doing so, please post to Teams and we will offer suggestions.
- **IMPORTANT:** When answering the numbered questions below, make sure to **label your answer with the question number**.  This will help us give appropriate grades.
  - EXAMPLE: "**Question 2A:** The Total Number of Itemsets is 19"
- **IMPORTANT:** **Show your work** and **explain it**.  This will help us give partial credit in some cases.

### Collaboration
For this assignment, you should work as an individual. You may informally discuss ideas with classmates, but your work should be your own.

### What You Need to Turn In
- Submit this Jupyter Notebook in .IPYNB format.  Do not "zip" the file.

### Notes
- Feel free to use the **mlxtend** package throughout this assignment.
- If a question asks you to "calculate" the number of "all possible rules", etc., explain the calculation by showing the "formula" you used. This will act as "showing your work".
***

***
# Section: 1 - Get Ready
1A) Load the data, and get it ready for association analysis. Do this with convenient python helper methods as appropriate. Feel free to use the tools given in the example we covered. 
- Suggest: Make the data one-hot encoded.
***

In [1]:
#Note: If the mlxtend library is not installed, uncomment the following line (once) and run it.
#!pip install mlxtend
import numpy as np
import pandas as pd
import mlxtend as mlx
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

In [2]:
#load the dataset
transaction_df = pd.read_excel('AssociationsGroceriesDataV1.xlsx',sheet_name = 'Transactions')
item_df = pd.read_excel('AssociationsGroceriesDataV1.xlsx',sheet_name = 'Items')

### One hot Encoding

In [3]:
# from mlxtend.preprocessing import TransactionEncoder

# te = mlx.preprocessing.TransactionEncoder()

# a = [[y for y in x if pd.notna(y)] for x in transaction_df.values.tolist()]
# mainList=[];
# list=[];
# idx=a[0][0]

# for cur,i in enumerate(a):
#     if(idx == i[0]):
#         list.append(i[1])
#     else:
#         idx=i[0]
#         mainList.append(list)
#         list=[]
#         list.append(i[1])
#     if(cur == len(a)-1):
#         mainList.append(list)
# te_array = te.fit(mainList).transform(mainList)
# ohe_df= pd.DataFrame(te_array, columns=te.columns_)

from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict
 
transaction_items = defaultdict(list)
for transaction in transaction_df.values.tolist():
    transaction_items[transaction[0]].append(transaction[1])
    
dataset_modified = list(transaction_items.values())
te = TransactionEncoder()

te_ary = te.fit(dataset_modified).transform(dataset_modified)
te_ary
 
ohe_df= pd.DataFrame(te_ary, columns=te.columns_)

In [60]:
ohe_df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False
9831,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9833,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### **Item information**

In [5]:
item_df

,ItemName,ItemNumber,UnitPrice
0,abrasive cleaner,1,11.58
1,artif. sweetener,2,19.36
2,baby cosmetics,3,12.45
3,baby food,4,16.80
4,bags,5,15.13
...,...,...,...
164,white bread,165,9.69
165,white wine,166,7.28
166,whole milk,167,5.22
167,yogurt,168,16.35


***
# Section: 2 - Explore the Data
***

***
## Section: 2.1 - Get the Transaction and Item Sizes
- Calculate the **number_of_transactions** and **number_of_items**.
***

In [53]:
num_of_transactions = len(ohe_df)
num_of_items = len(item_df["ItemName"])
print("Section:2.1 Number of transactions: ", num_of_transactions)
print("Section:2.1 Number of Items: ", num_of_items)


Section:2.1 Number of transactions:  9835
Section:2.1 Number of Items:  169


***
## Section: 2.2 - Evaluate the Itemset and Rule Size & Complexity
- Calculate the **maximum number of Itemsets** that could be created from the items (without considering the actual transaction data). Show your work.
- Calculate the **maximum number of Rules** that can be created from the items (without considering the actual transaction data). Show your work.
- What do the calculations suggest as a **potential cause of concern**? Hint: Complexity.
- What might you do to manage these concerns?
***

In [7]:
#2^d where d is number of items
max_itemsets = pow(2, num_of_items)
print("Section 2.2 Maximum number of itemsets: " , max_itemsets)

#3^d - 2^(d-1) + 1 where d is number of items
max_rules = pow(3, num_of_items) - pow(2, num_of_items+1) + 1
print("Section 2.2 Maximum number of Rules", max_rules)

Maximum number of itemsets:  748288838313422294120286634350736906063837462003712
Maximum number of Rules 430023359390034222082732011946860220634520402626757122001337339969404822623413860


#### Problem
The potential cause of concern is number of itemsets and max rules is increasing exponentially as the number of item increases where it increases the complensity of finding the candidate rules that we can make. The complexity of itemsets is O(NMw) where N is number of transaction, m is number of itemsets and w is number of items in one transaction. 

#### Possible Solutions
1. we can reduce the number of candidates, that is number of itemsets
2. we can reduce the size of transactions
3. We can reduce the number of comparison by not comparing every candidates with the transaction. 

***
# Section: 3 - Itemset Generation
***

***
## Section: 3.1 - Revise the Dataset
- If/as appropriate, trim or revise the dataset to make the runtime reasonable.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [8]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets_ap = mlx.frequent_patterns.apriori(ohe_df, min_support=.01, use_colnames=True)

In [9]:
sorted_itemsets = frequent_itemsets_ap.sort_values("support", ascending=False)
sorted_itemsets

,support,itemsets
86,0.255516,(whole milk)
55,0.193493,(other vegetables)
66,0.183935,(rolls/buns)
75,0.174377,(soda)
87,0.139502,(yogurt)
...,...,...
178,0.010066,"(sausage, frankfurter)"
306,0.010066,"(curd, whole milk, yogurt)"
160,0.010066,"(curd, rolls/buns)"
212,0.010066,"(tropical fruit, napkins)"


### Explanation

In section 3.1, I used apriori algorithm to trim down the dataset. Apriori algorithm uses minimum support to cut off the least frequent subset from list to find rule more easier. 

In the code, I sorted the itemsets with the descending order, meaning that the highest support itemset will be placed at the top and lowest at the bottom. I set the min-support to 0.01, meaning that appearing one out of 100 will be be used. Thus, it means that whole milk is most frequently bought item and hard cheese, whole milk is a least likely to bought.


***
## Section: 3.2 - Create Two-Itemsets
- Create a set of 20 two-item sets with highest support. Sort them in decreasing order of support.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [10]:
# two_itemsets=[]
# idx=0

# for i, j in zip(sorted_itemsets["support"], sorted_itemsets["itemsets"]):
#     list=[]
    
#     if(len(j) == 2):
#         list.append(i)
#         list.append(j)
#         tmp=[]
        
#         two_itemsets.append(list)
# top_itemsets = two_itemsets[:20]
# top_df = pd.DataFrame(top_itemsets)
# top_df.columns=['support', 'itemsets']
# top_df

top_df = sorted_itemsets.loc[sorted_itemsets["itemsets"].str.len() == 2]
top_df[:20]

,support,itemsets
239,0.074835,"(whole milk, other vegetables)"
267,0.056634,"(rolls/buns, whole milk)"
300,0.056024,"(whole milk, yogurt)"
274,0.048907,"(whole milk, root vegetables)"
229,0.047382,"(other vegetables, root vegetables)"
240,0.043416,"(yogurt, other vegetables)"
228,0.042603,"(rolls/buns, other vegetables)"
294,0.042298,"(tropical fruit, whole milk)"
290,0.040061,"(soda, whole milk)"
264,0.038332,"(rolls/buns, soda)"


### Explanation

Through result, we can see that (other vegetables, whole milk) has the highest support and (while milk, demestic eggs) has lowest support. It means that (whole milk, domestic eggs) are likely to be bought together and (whole milk and domestic eggs) is least likely bought together.

with the highest support at the top, I placed the top 20 support from two itemset list. In the code, I used the sorted list of items with support values. Taking only itemset with length 2 into new list of itemset with its support value. Then, I listed two itemsets with top 20 support values

***
# Section: 4 - Generate Rules
- For the two-itemsets created above, create the related rules.
***

In [11]:
rules_ap = mlx.frequent_patterns.association_rules(sorted_itemsets, metric="confidence", min_threshold=0.1)

In [12]:
result = rules_ap.loc[rules_ap["antecedents"].str.len() == 1]
result2=result[:40]
result2.sort_values("antecedent support", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(whole milk),(other vegetables),0.255516,0.193493,0.074835,0.292877,1.513634,0.025394,1.140548,0.455803
38,(whole milk),(domestic eggs),0.255516,0.063447,0.029995,0.117390,1.850203,0.013783,1.061117,0.617231
35,(whole milk),(citrus fruit),0.255516,0.082766,0.030503,0.119379,1.442377,0.009355,1.041577,0.411963
37,(whole milk),(pip fruit),0.255516,0.075648,0.030097,0.117788,1.557043,0.010767,1.047765,0.480544
26,(whole milk),(pastry),0.255516,0.088968,0.033249,0.130123,1.462587,0.010516,1.047312,0.424831
22,(whole milk),(bottled water),0.255516,0.110524,0.034367,0.134501,1.216940,0.006126,1.027703,0.239450
17,(whole milk),(soda),0.255516,0.174377,0.040061,0.156785,0.899112,-0.004495,0.979136,-0.130978
15,(whole milk),(tropical fruit),0.255516,0.104931,0.042298,0.165539,1.577595,0.015486,1.072631,0.491782
31,(whole milk),(whipped/sour cream),0.255516,0.071683,0.032232,0.126144,1.759754,0.013916,1.062323,0.579917
6,(whole milk),(root vegetables),0.255516,0.108998,0.048907,0.191405,1.756031,0.021056,1.101913,0.578298


In [13]:
result2['antecedents'].value_counts()

(whole milk)            12
(other vegetables)       6
(rolls/buns)             5
(yogurt)                 3
(soda)                   3
(root vegetables)        2
(tropical fruit)         2
(bottled water)          1
(pastry)                 1
(whipped/sour cream)     1
(sausage)                1
(citrus fruit)           1
(pip fruit)              1
(domestic eggs)          1
Name: antecedents, dtype: int64

***
# Section: 5 - Rule Evaluation
- For the rules created above, find the single Item (that would be given the discount) that would cause the greatest increase in monthly store revenue.
  - This is based on the Business Problem stated at the top of this notebook.
  - Consider:
    - How much will the store's monthly revenue decrease (or increase) due to the change in price for the chosen Item (and its increased sales)?
    - How much will the store's monthly revenue increase (or decrease) due to the increased sales of the associated Items?
***

In [79]:
def itemBought(bought):
    num=0
    for transaction in dataset_modified:
        for item in transaction:
            if(item == bought):
                num+=1
    return num

This function calculate the number of times that customer bought the certain item.

Based on the result above, I will choose the whole milk as the single item that can increase monthly store revenue. It is the item that affect 12 other items. Thus, it has the highest support as the result of apriori algorithm. 

Price of the whole milke is 5.22. So, we will decrease the price by 10% which is 0.52. 5.22 - 0.52 is $ 4.7.

In [80]:
#milk_df = sorted_itemsets.loc[sorted_itemsets["itemsets"].astype(str).str.contains("whole milk")]
#milk_num = len(milk_df)

milk_num=itemBought("whole milk")
            
print("Number of milk bought: ", milk_num) 
org_revenue_milk = milk_num * 5.22
new_revenue_milk = milk_num * 1.05 * 4.7
revenue_decrease =  org_revenue_milk - new_revenue_milk
print("The original month revenue by milk: $", org_revenue_milk)
print("The month revenue by milk after sale: $", new_revenue_milk)
print("The monthly revenue decrease: -$",  revenue_decrease)

Number of milk bought:  2513
The original month revenue by milk: $ 13117.859999999999
The month revenue by milk after sale: $ 12401.655
The monthly revenue decrease: -$ 716.2049999999981


Using the number from above calculation, I made calculation from past transaction. There were 2513 whole milk bounght. Then, we assume that decrease the price will encourage customer to purchage 5% more of this item. So, 2513 * 1.05 will be expected item numbers and multiply this number by 4.7, which is sales price.

Thus, I got new monthly revenue of $ 12401.655

We can see the original revenue before sale was $13117.86.

Total revenue decrease by about $-716.20.

In [73]:
result3 = result2.loc[result2["antecedents"].astype(str).str.contains("whole milk")]
result4 = result3["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

itemNum=[]
tmp=[]
org_revenue=0
new_revenue=0
for item in result4:
    tmp=[]
    #org_qty =len(sorted_itemsets.loc[sorted_itemsets["itemsets"].astype(str).str.contains(item)])
    
    org_qty=itemBought(item)
    
    new_qty = org_qty * 1.05
    price=float(item_df.loc[item_df["ItemName"].astype(str).str.contains(item)]["UnitPrice"].to_string(index=False))
    
    org_revenue += price * org_qty
    new_revenue += price * new_qty
    tmp.append(item)
    tmp.append(org_qty)
    tmp.append(price)
    itemNum.append(tmp)

itemNum

[['other vegetables', 1903, 18.99],
 ['rolls/buns', 1809, 9.29],
 ['yogurt', 1372, 16.35],
 ['root vegetables', 1072, 14.26],
 ['tropical fruit', 1032, 6.71],
 ['soda', 1715, 13.88],
 ['bottled water', 1087, 12.83],
 ['pastry', 875, 9.4],
 ['whipped/sour cream', 705, 19.26],
 ['citrus fruit', 814, 9.26],
 ['pip fruit', 744, 8.21],
 ['domestic eggs', 624, 5.29]]

In [77]:
print("Original Revenue before sale: $",org_revenue)
print("New Revenue after sale: $",new_revenue)
print("Net Benefit: $", new_revenue-org_revenue)
print("Total increase in revenue", new_revenue-org_revenue-revenue_decrease)

Original Revenue before sale: $ 174087.76999999993
New Revenue after sale: $ 182792.1585
Net Benefit: $ 8704.38850000006
Total increase in revenue 7988.183500000061


The list above cotains the name of the items that are affected by the whole milk ,number of times that they are bought, and their price. 

Then, in business problem, they are effected to have 5% more item bought from sales item. So, using the same calcuation as above, itemNumber * 1.05 * Price to get the new expected income. 

So, from calculation, we can see the orginal revenue was $174087.77

The new revenue increased to $182792.16

So, net benefit is $8704.39

Subtracting this number to  decrease revenue by sales on whole milk, which is $ 716.20

We get total about $7988.18 more revenue. 

***
# Section: 6 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

From this homework, I learn how to handle the associate analysis with real time example. I am able to list the items in order of support to get the most appropriate item that I needed. Then, I handled this list to get result that I wanted. Furthermore, I learn how to handle the dataframe more correctly. Previously, I used the dataframe with loop. However, I begin to use some functions of the dataframe to have more efficiency on handling the items. 

***
### END-OF-SUBMISSION
***